In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
# imports
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [ ]:
churn_data = pd.read_csv('./data/WA_Fn-UseC_-Telco-Customer-Churn.csv')

In [24]:
N = churn_data.shape[0]
churned = len(churn_data.query("Churn == 'Yes'"))
notChurned = len(churn_data.query("Churn == 'No'"))

print(f'customers: {N}\n')
print(f"customers who churned: {churned}")
print(f"customers who haven't churned yet: {notChurned}\n")
print(f'percentage of customers who churned: {100*churned/len(churn_data):.0f}%')
print(f"percentage of customers who haven't churned yet: {100*notChurned/len(churn_data):.0f}%")

customers: 7043

customers who churned: 1869
customers who haven't churned yet: 5174

percentage of customers who churned: 27%
percentage of customers who haven't churned yet: 73%


In [5]:
# Drop customerID column and TotalCharges column
# otherwise together with MonthlyCharges we can deduce how many months someone have been subscribed
churn_data = churn_data.drop(['customerID', 'TotalCharges'], axis=1)

In [6]:
churn_data.head().T

,0,1,2,3,4
gender,Female,Male,Male,Male,Female
SeniorCitizen,0,0,0,0,0
Partner,Yes,No,No,No,No
Dependents,No,No,No,No,No
tenure,1,34,2,45,2
PhoneService,No,Yes,Yes,No,Yes
MultipleLines,No phone service,No,No,No phone service,No
InternetService,DSL,DSL,DSL,DSL,Fiber optic
OnlineSecurity,No,Yes,Yes,Yes,No
OnlineBackup,Yes,No,Yes,No,No


In [7]:
from sksurv.datasets import get_x_y

X, y = get_x_y(churn_data, attr_labels=['Churn','tenure'], pos_label='Yes')

In [8]:
X.head().T

,0,1,2,3,4
gender,Female,Male,Male,Male,Female
SeniorCitizen,0,0,0,0,0
Partner,Yes,No,No,No,No
Dependents,No,No,No,No,No
PhoneService,No,Yes,Yes,No,Yes
MultipleLines,No phone service,No,No,No phone service,No
InternetService,DSL,DSL,DSL,DSL,Fiber optic
OnlineSecurity,No,Yes,Yes,Yes,No
OnlineBackup,Yes,No,Yes,No,No
DeviceProtection,No,Yes,No,Yes,No


In [9]:
y[:5]

array([(False,  1.), (False, 34.), ( True,  2.), (False, 45.),
       ( True,  2.)], dtype=[('Churn', '?'), ('tenure', '<f8')])

In [10]:
from sklearn.model_selection import train_test_split

X_trn, X_val, y_trn, y_val = train_test_split(X, y, random_state=42)

In [11]:
print(f'Number of training samples: {len(y_trn)}')
print(f'Number of validation samples: {len(y_val)}')

Number of training samples: 5282
Number of validation samples: 1761


In [12]:
scaling_cols = ['MonthlyCharges']
scaling_cols

['MonthlyCharges']

In [13]:
categorical_cols = ['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'PhoneService',
        'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup',
        'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies',
        'Contract', 'PaperlessBilling', 'PaymentMethod']
categorical_cols

['gender',
 'SeniorCitizen',
 'Partner',
 'Dependents',
 'PhoneService',
 'MultipleLines',
 'InternetService',
 'OnlineSecurity',
 'OnlineBackup',
 'DeviceProtection',
 'TechSupport',
 'StreamingTV',
 'StreamingMovies',
 'Contract',
 'PaperlessBilling',
 'PaymentMethod']

In [14]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler


preprocessor = ColumnTransformer(
    [('cat-preprocessor', OneHotEncoder(drop='first'), categorical_cols),
    ('standard-scaler', StandardScaler(), scaling_cols)],
    remainder='passthrough', sparse_threshold=0)

In [15]:
X_trn = preprocessor.fit_transform(X_trn)
X_val = preprocessor.transform(X_val)

In [16]:
from sksurv.linear_model import CoxPHSurvivalAnalysis
from sksurv.metrics import concordance_index_censored

cph = CoxPHSurvivalAnalysis(alpha=0.1)
cph.fit(X_trn, y_trn)
ci_cph = concordance_index_censored(y_val["Churn"], y_val["tenure"], cph.predict(X_val))
print(f'The c-index of Cox PH is given by {ci_cph[0]:.3f}')

The c-index of Gradient Boosting Cox PH is given by 0.865


In [20]:
print("The hyper-parameters for Cox PH are:")
for param_name in cph.get_params().keys():
    print(param_name)

The hyper-parameters are for the full-pipeline are:
alpha
n_iter
ties
tol
verbose


In [21]:
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV

param_distributions = {
    'alpha': reciprocal(0.1, 100),
}

model_random_search = RandomizedSearchCV(
    cph, param_distributions=param_distributions, n_iter=50, n_jobs=-1, cv=5, random_state=42)
model_random_search.fit(X_trn, y_trn)

print(
    f"The c-index of Cox using a {model_random_search.__class__.__name__} is "
    f"{model_random_search.score(X_val, y_val):.3f}")
print(
    f"The best set of parameters is: {model_random_search.best_params_}"
)

The c-index of Cox using a RandomizedSearchCV is 0.865
The best set of parameters is: {'alpha': 0.7523742884534856}


In [18]:
from sksurv.ensemble import GradientBoostingSurvivalAnalysis

gbc = GradientBoostingSurvivalAnalysis(random_state=42)
gbc.fit(X_trn, y_trn)
ci_gbc = concordance_index_censored(y_val["Churn"], y_val["tenure"], gbc.predict(X_val))
print(f'The c-index of Gradient Boosting Cox PH is given by {ci_gbc[0]:.3f}')

The c-index of Gradient Boosting Cox PH is given by 0.853


In [22]:
print("The hyper-parameters for Gradient Boosting Cox PH are:")
for param_name in gbc.get_params().keys():
    print(param_name)

The hyper-parameters are for the full-pipeline are:
criterion
dropout_rate
learning_rate
loss
max_depth
max_features
max_leaf_nodes
min_impurity_decrease
min_impurity_split
min_samples_leaf
min_samples_split
min_weight_fraction_leaf
n_estimators
presort
random_state
subsample
verbose


In [23]:
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV

class reciprocal_int:
    def __init__(self, a, b):
        self._distribution = reciprocal(a, b)

    def rvs(self, *args, **kwargs):
        return self._distribution.rvs(*args, **kwargs).astype(int)

param_distributions = {
    'learning_rate': reciprocal(0.001, 1),
    'max_depth': reciprocal_int(5, 50),
    'min_samples_leaf': reciprocal_int(1, 40),
}

model_random_search = RandomizedSearchCV(
    gbc, param_distributions=param_distributions, n_iter=50, n_jobs=-1, cv=5, random_state=42)
model_random_search.fit(X_trn, y_trn)

print(
    f"The c-index of gradient boosting Cox PH using a {model_random_search.__class__.__name__} is "
    f"{model_random_search.score(X_val, y_val):.3f}")
print(
    f"The best set of parameters is: {model_random_search.best_params_}"
)

The c-index of gradient boosting Cox PH using a RandomizedSearchCV is 0.859
The best set of parameters is: {'learning_rate': 0.13311216080736882, 'max_depth': 5, 'min_samples_leaf': 35}


In [17]:
from pysurvival.models.survival_forest import RandomSurvivalForestModel

rsf = RandomSurvivalForestModel()
rsf.fit(X_trn, y_trn["tenure"], y_trn["Churn"], seed=42)
ci_rsf = concordance_index_censored(y_val["Churn"], y_val["tenure"], rsf.predict_risk(X_val))
print(f'The c-index of Random survival forest is given by {ci_rsf[0]:.3f}')

The c-index of Random survival forest is given by 0.830


In [19]:
from pysurvival.models.semi_parametric import NonLinearCoxPHModel

structure = [ {'activation': 'ReLU', 'num_units': 50}, {'activation': 'ReLU', 'num_units': 10}, ]

# Building the model
nonlinear_coxph = NonLinearCoxPHModel(structure=structure)
nonlinear_coxph.fit(X_trn, y_trn["tenure"], y_trn["Churn"])
ci_deepsurv = concordance_index_censored(y_val["Churn"], y_val["tenure"], nonlinear_coxph.predict_risk(X_val))
print(f'The c-index of DeepSurv is given by {ci_deepsurv[0]:.3f}')

The c-index of DeepSurv is given by 0.857
